In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier

In [3]:
order_data = pd.read_csv("./training_data/order_data/order_data_2016-01-01", header=None, delim_whitespace=True)
for i in range(2, 22):
    order_data = order_data.append(pd.read_csv("./training_data/order_data/order_data_2016-01-%02d" % i, header=None, delim_whitespace=True))
cluster_map = pd.read_csv("./training_data/cluster_map/cluster_map", header=None, delim_whitespace=True)

In [4]:
order_data.count

<bound method DataFrame.count of                                        0                                 1  \
0       97ebd0c6680f7c0535dbfdead6e51b4b  dd65fa250fca2833a3a8c16d2cf0457c   
1       92c3ac9251cc9b5aab90b114a1e363be  c077e0297639edcb1df6189e8cda2c3d   
2       abeefc3e2aec952468e2fd42a1649640  86dbc1b68de435957c61b5a523854b69   
3       cb31d0be64cda3cc66b46617bf49a05c  4fadfa6eeaa694742de036dddf02b0c4   
4       139d492189ae5a933122c098f63252b3                               NaN   
5       b0b59fd0fe98bf603972da2f62e6522d                               NaN   
6       17c1c85144ab532947c7ea724fdcc945  115ac9b23f00a2e6d8a3041e23469f41   
7       d682c1c004024f8937d21cd43498d1bb  7cb7ccbc192c177b136feca274fb3d07   
8       6fcae38baf2eb52e17273df41bf6fc6f  c586e492c4587c3aa15557a8eb41806d   
9       70afc52be8a6d35137f2277a6ca88017  ea61e970ba415f946b3b35464c74f32a   
10      29251671fcc9d9078b760ce3f6f7994b                               NaN   
11      ef10162b38fadcd75a5751b

In [5]:
order_data["time_slice"] = order_data[7].apply(lambda x: (int(x.split(':')[0]) * 60 * 60 + int(x.split(':')[1]) * 60 + int(x.split(':')[2])) / 600)

In [6]:
order_data['time_slice'].max()

143

In [7]:
region_mapping = {}
for item in cluster_map.values:
    region_mapping[item[0]] = item[1]

In [8]:
order_data["region_id"] = order_data[3].apply(lambda x: int(region_mapping[x]))

In [9]:
order_data["date"] = order_data[6].apply(lambda x: int(x.split('-')[2]))

In [10]:
print order_data['time_slice'].min()
print order_data['time_slice'].max()
print order_data['region_id'].min()
print order_data['region_id'].max()

0
143
1
66


In [11]:
demand_count = {}
supply_count = {}
for i in range(1, 22):
    for j in range(0, 144):
        demand_count[(i - 1) * 144 + j] = {}
        supply_count[(i - 1) * 144 + j] = {}
        for z in range(1, 67):
            demand_count[(i - 1) * 144 + j][z] = 0
            supply_count[(i - 1) * 144 + j][z] = 0

In [12]:
order_data.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, u'time_slice', u'region_id', u'date'], dtype='object')

In [56]:
for item in order_data.values:
    demand_count[(item[10] - 1) * 144 + item[8]][item[9]] += 1
    if pd.isnull(item[1]) == False:
        supply_count[(item[10] - 1) * 144 + item[8]][item[9]] += 1

In [58]:
item = order_data.values[4]
pd.isnull(item[1])

True

In [59]:
training_data = pd.DataFrame(columns = ["region_id", "time_slice_id", "gap_1", "gap_2", "gap_3", "gap"])

In [60]:
training_data

,region_id,time_slice_id,gap_1,gap_2,gap_3,gap


In [61]:
for i in range(1, 67):
    for j in range(4, 144 * 21):
        tsi = j % 144
        training_data = training_data.append({"region_id" : i, "time_slice_id" : tsi, "gap_1" : abs(demand_count[j - 1][i] - supply_count[j - 1][i]), "gap_2" : abs(demand_count[j - 2][i] - supply_count[j - 2][i]), "gap_3" : abs(demand_count[j - 3][i] - supply_count[j - 3][i]), "gap" : abs(demand_count[j][i] - supply_count[j][i])}, ignore_index=True)
        

In [71]:
training_data = training_data.astype(float)

In [78]:
from sklearn.ensemble import RandomForestRegressor
target = training_data["gap"]
train = training_data.drop("gap", 1)

In [80]:
rfc = RandomForestRegressor()
rfc.fit(train, target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [83]:
test = pd.read_csv("test.csv", header=None)

In [85]:
result = test.copy()

In [86]:
result.head()

,0,1,2
0,1,2016-01-22-46,0
1,2,2016-01-22-46,0
2,3,2016-01-22-46,0
3,4,2016-01-22-46,0
4,5,2016-01-22-46,0


In [87]:
test_order_data = pd.read_csv("./test_set_1/order_data/order_data_2016-01-22_test", header=None, delim_whitespace=True)
for i in [24, 26, 28, 30]:
    test_order_data = test_order_data.append(pd.read_csv("./test_set_1/order_data/order_data_2016-01-%02d_test" % i, header=None, delim_whitespace=True))

In [88]:
test_order_data.head()

,0,1,2,3,4,5,6,7
0,e37f842c2a37de68e16466a3c56b916b,NaN,5903295e07afb893e1a0f7073965515d,b05379ac3f9b7d99370d443cfd5dcc28,82cc4851f9e4faa4e54309f8bb73fd7c,7.0,2016-01-22,17:19:43
1,62588e55ff8892ba38a0bbe5678be272,53919c82b12bd39b12d77d4d8db1dda1,5903295e07afb893e1a0f7073965515d,b05379ac3f9b7d99370d443cfd5dcc28,82cc4851f9e4faa4e54309f8bb73fd7c,12.0,2016-01-22,17:19:43
2,693194e4d57cdd500e793c0c1e4f7a93,19f5ba02d33855688b727e39c98b2939,dbea2d56b9fcacf8b18c40a4f34fbbf2,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,18.0,2016-01-22,11:00:05
3,3a0c651d2558d9083a66fc179e3ba81c,NaN,4b9957e8fb4d8cf981560e2a512aa598,d4ec2125aff74eded207d2d915ef682f,d1ab2cc538d518758a1a82b1787592d4,36.0,2016-01-22,17:00:42
4,68b7cec210c7f875b79ce32dab7195ea,NaN,4b9957e8fb4d8cf981560e2a512aa598,d4ec2125aff74eded207d2d915ef682f,d1ab2cc538d518758a1a82b1787592d4,36.0,2016-01-22,17:00:42
